In [1]:
import pandas as pd
import numpy as np
import pickle
from collections import defaultdict
import logging

from utils import amount_to_float

In [2]:
with open('./data/pool_data_raw.pickle', 'rb') as f:
    data = pickle.load(f)

In [3]:
sorted(list(data.keys()))

['DAI-WETH-3000',
 'DAI-WETH-500',
 'FRAX-WETH-3000',
 'USDC-WETH-3000',
 'USDC-WETH-500',
 'WBTC-USDC-3000',
 'WETH-USDT-3000',
 'WETH-USDT-500']

In [4]:
for pool, dset in data.items():
    if dset['swaps'] is None:
        print(f"{pool} missing swap data.")
    if dset['liquidity'] is None:
        print(f"{pool} missing liquidity data.")

In [5]:
for pool, dset in data.items():
    try:
        swap_pool_addr = dset['swaps'].at[0, 'pool_address']
        liqu_pool_addr = dset['liquidity'].at[0, 'pool_address']
        if swap_pool_addr != liqu_pool_addr:
            print(f"{pool} has mismatching pool addresses: {swap_pool_addr}, {liqu_pool_addr}")
    except AttributeError:
        pass

In [6]:
for pool, dset in data.items():
    for dtype, df in dset.items():
        # supports_erc_token_{0,1} is a list value, which is not hashable,
        # so we have to exclude it from duplicated
        cols = [c for c in df.columns if not ('supports_erc_token' in c)]
        n_dups = df.duplicated(subset=cols).sum()
        print(f"{dtype} for {pool} has {n_dups:,.0f} duplicates to remove.")
        
        df.drop_duplicates(subset=cols, inplace=True, ignore_index=True)
        print(f"{df.shape[0]:,.0f} transactions remaining.")
        
    print('')  # lazy way to do a new line 

swaps for FRAX-WETH-3000 has 0 duplicates to remove.
22 transactions remaining.
liquidity for FRAX-WETH-3000 has 0 duplicates to remove.
10 transactions remaining.

swaps for WETH-USDT-500 has 253 duplicates to remove.
735,956 transactions remaining.
liquidity for WETH-USDT-500 has 40 duplicates to remove.
6,378 transactions remaining.

swaps for USDC-WETH-500 has 597 duplicates to remove.
1,247,617 transactions remaining.
liquidity for USDC-WETH-500 has 207 duplicates to remove.
31,210 transactions remaining.

swaps for WBTC-USDC-3000 has 0 duplicates to remove.
59,869 transactions remaining.
liquidity for WBTC-USDC-3000 has 37 duplicates to remove.
10,876 transactions remaining.

swaps for DAI-WETH-500 has 2 duplicates to remove.
295,895 transactions remaining.
liquidity for DAI-WETH-500 has 51 duplicates to remove.
3,464 transactions remaining.

swaps for WETH-USDT-3000 has 1 duplicates to remove.
159,300 transactions remaining.
liquidity for WETH-USDT-3000 has 203 duplicates to rem

In [7]:
for pool, dset in data.items():
    min_swap = dset['swaps']['swap_time'].min()
    max_swap = dset['swaps']['swap_time'].max()
    n_swap = dset['swaps'].shape[0]
    
    min_liqu = dset['liquidity']['txn_time'].min()
    max_liqu = dset['liquidity']['txn_time'].max()
    n_liqu = dset['liquidity'].shape[0]
    
    print(f'Pool:            {pool}')
    print(f'Swap Range:      {min_swap} to {max_swap}, n={n_swap:,.0f}')
    print(f'Liquidity Range: {min_liqu} to {max_liqu}, n={n_liqu:,.0f}\n')

Pool:            FRAX-WETH-3000
Swap Range:      2021-05-19T10:37:08Z to 2021-06-30T12:29:16Z, n=22
Liquidity Range: 2021-05-06T03:25:59Z to 2021-07-09T09:36:17Z, n=10

Pool:            WETH-USDT-500
Swap Range:      2021-05-05T22:23:23Z to 2022-01-29T23:41:35Z, n=735,956
Liquidity Range: 2021-05-05T21:46:12Z to 2022-01-29T13:45:44Z, n=6,378

Pool:            USDC-WETH-500
Swap Range:      2021-05-05T22:15:01Z to 2022-01-30T04:52:19Z, n=1,247,617
Liquidity Range: 2021-05-05T21:42:11Z to 2022-01-29T18:52:48Z, n=31,210

Pool:            WBTC-USDC-3000
Swap Range:      2021-05-05T21:08:03Z to 2022-01-29T19:16:46Z, n=59,869
Liquidity Range: 2021-05-05T19:13:15Z to 2022-01-29T14:34:45Z, n=10,876

Pool:            DAI-WETH-500
Swap Range:      2021-05-05T18:09:08Z to 2022-01-29T19:26:20Z, n=295,895
Liquidity Range: 2021-05-05T18:03:43Z to 2022-01-28T04:12:04Z, n=3,464

Pool:            WETH-USDT-3000
Swap Range:      2021-05-05T17:52:33Z to 2022-01-29T19:38:25Z, n=159,300
Liquidity Range: 20

In [8]:
def parse_liquidity_logs(liquidity_txns, print_checks=False):
    data = defaultdict(set)
    for tx_hash, txn in liquidity_txns.items():
        for log in txn['log_events']:
            if log['decoded'] is None:
                continue

            name = log['decoded']['name']
            params = log['decoded']['params']
            if params is None:
                continue

            incr_or_decr = (name == 'IncreaseLiquidity') or (name == 'DecreaseLiquidity')
            mint_or_burn = (name == 'Mint') or (name == 'Burn')
            if (not incr_or_decr) and (not mint_or_burn):
                continue

            amount0, amount1, liquidity = None, None, None
            tick_lower, tick_upper = None, None
            for entry in params:
                if entry['name'] == 'amount0':
                    amount0 = entry['value']
                if entry['name'] == 'amount1':
                    amount1 = entry['value']
                if entry['name'] == 'tickLower':
                    tick_lower = entry['value']
                if entry['name'] == 'tickUpper':
                    tick_upper = entry['value']

                if (entry['name'] == 'liquidity') and incr_or_decr:
                    liquidity = entry['value']
                elif (entry['name'] == 'amount') and mint_or_burn:
                    liquidity = entry['value']
    
            if print_checks:
                any_nones = (
                    (amount0 is None) or 
                    (amount1 is None) or 
                    (liquidity is None) or
                    (tick_lower is None) or
                    (tick_upper is None)
                )
                if any_nones:
                    print(f'Check {name}, {tx_hash}')

            data[tx_hash].add((name, amount0, amount1, liquidity, tick_lower, tick_upper))
    
    return data

In [9]:
with open('./data/liquidity_transactions.pickle', 'rb') as f:
    liquidity_txns = pickle.load(f)

In [10]:
liquidity_data = {}
for pool, dset in data.items():
    liquidity_data[pool] = parse_liquidity_logs(liquidity_txns[pool])
    
    n1 = dset['liquidity']['tx_hash'].nunique()
    n2 = len(liquidity_data[pool].keys())
    print(f'Pool: {pool}')
    print(f'Transaction Hashes: {n1:,} vs. {n2:,}\n')

Pool: FRAX-WETH-3000
Transaction Hashes: 10 vs. 10

Pool: WETH-USDT-500
Transaction Hashes: 5,496 vs. 5,496

Pool: USDC-WETH-500
Transaction Hashes: 28,220 vs. 28,220

Pool: WBTC-USDC-3000
Transaction Hashes: 7,889 vs. 7,889

Pool: DAI-WETH-500
Transaction Hashes: 3,160 vs. 3,160

Pool: WETH-USDT-3000
Transaction Hashes: 22,629 vs. 22,629

Pool: USDC-WETH-3000
Transaction Hashes: 65,047 vs. 65,047

Pool: DAI-WETH-3000
Transaction Hashes: 9,881 vs. 9,881



In [11]:
def find_matching_txn(liquidity_data, tx_hash, token0, token1, event_type, 
                      tick_lower, tick_upper):
    txn = liquidity_data[tx_hash]
    matched_entry = None  # for the some of the logic around the mints/burns
     
    for entry in txn:
        if event_type == 'ADD_LIQUIDITY':
            name_match = (entry[0] == 'IncreaseLiquidity') or (entry[0] == 'Mint')
            token0_match = (entry[1] == token0)
            token1_match = (entry[2] == token1)
            tick_lower_match = (entry[4] == tick_lower)
            tick_upper_match = (entry[5] == tick_upper)
            if name_match and token0_match and token1_match and tick_lower_match and tick_upper_match:
                # some mints and burns are missing liquidity data
                # rather than immediately returning such entries, we continue to search 
                # through the entries to see if another matching entry with liquidity exists
                if (entry[3] is None) or (entry[3] == 0):
                    matched_entry = entry
                else:
                    return entry
            
        elif event_type == 'REMOVE_LIQUIDITY':
            name_match = (entry[0] == 'DecreaseLiquidity') or (entry[0] == 'Burn')
            # REMOVE_LIQUIDITY transactions have an issue where token0 is actually token1 
            # and token1 is always 0
            token1_match = (entry[2] == token0) 
            tick_lower_match = (entry[4] == tick_lower)
            tick_upper_match = (entry[5] == tick_upper)
            if name_match and token1_match and tick_lower_match and tick_upper_match:
                # some mints and burns are missing liquidity data
                # rather than immediately returning such entries, we continue to search 
                # through the entries to see if another matching entry with liquidity exists
                if (entry[3] is None) or (entry[3] == 0):
                    matched_entry = entry
                else:
                    return entry
            
        else:
            raise ValueError(f'{event_type} is not a valid event_type.')
            
    return matched_entry

In [12]:
for pool, dset in data.items():
    dset['liquidity']['txn_data'] = dset['liquidity'].apply(
        lambda df: find_matching_txn(
            liquidity_data[pool], 
            df['tx_hash'], 
            df['token_0_amount'], 
            df['token_1_amount'], 
            df['liquidity_event'], 
            df['price_tick_lower'], 
            df['price_tick_upper'], 
        ), 
        axis=1
    )
    if dset['liquidity']['txn_data'].isna().sum() > 0:
        print(f'Could not match one or more transactions for {pool}.')
        
    cols = [
        'event_name',
        # update token_{0,1}_amount columns with the values from the logs
        # for adds, these will be the exact same, for removes these will be
        # the corrected values
        'token_0_amount',  
        'token_1_amount',
        'abs_liquidity',
        'tick_lower_DROP',
        'tick_upper_DROP'
    ]
    dset['liquidity'][cols] = pd.DataFrame(dset['liquidity']['txn_data'].tolist(), 
                                           index=dset['liquidity'].index)
    if dset['liquidity']['abs_liquidity'].isna().any():
        print(f'Null liquidity for one or more transactions for {pool}.')
        
    drop_cols = ['txn_data', 'tick_lower_DROP', 'tick_upper_DROP']
    dset['liquidity'].drop(drop_cols, axis=1, inplace=True)

In [13]:
def get_account_map(liquidity_txns):
    account_map = {}
    for tx_hash, data in liquidity_txns.items():
        account_map[tx_hash] = data['from_address']
        
    return account_map

for pool, dset in data.items():
    account_map = get_account_map(liquidity_txns[pool])
    dset['liquidity']['account_address'] = dset['liquidity']['tx_hash'].map(account_map)

In [14]:
def clean_data(df, pool_info):
    # get the index for the pool in the pool_info DataFrame
    pool_addr = df.at[0, 'pool_address']
    pool_idx = pool_info.index[pool_info['pool_contract_address'] == pool_addr][0]
    
    # validate that certain data matches the pool_info data
    val_cols = [
        'contract_ticker_symbol_token_0',
        'contract_ticker_symbol_token_1',
        'contract_decimals_token_0',
        'contract_decimals_token_1'
    ]
    for col in val_cols:
        assert df.at[0, col] == pool_info.at[pool_idx, col]
    
    # drop all null columns
    null_cols = [
        'dex_name', 
        'chain_name',
        'chain_id',
        'block_signed_at'
    ]
    for col in null_cols:
        assert df[col].isna().sum() == df.shape[0]
    df = df.drop(null_cols, axis=1)
    
    # drop unused columns
    unused_cols = [
        'logo_url_token_0',
        'logo_url_token_1',
        'supports_erc_token_0',
        'supports_erc_token_1'
    ]
    df = df.drop(unused_cols, axis=1)
    
    # convert token_{0,1}_amount to floats using the required decimals
    token0_decimals = df.at[0, 'contract_decimals_token_0']
    token1_decimals = df.at[0, 'contract_decimals_token_1']
    df['token_0_amount'] = df['token_0_amount'].map(lambda x: amount_to_float(x, token0_decimals))
    df['token_1_amount'] = df['token_1_amount'].map(lambda x: amount_to_float(x, token1_decimals))
    
    # add relevant columns from pool_info
    df['pool_fee'] = pool_info.at[pool_idx, 'pool_fee']
    df['pool_tick_spacing'] = pool_info.at[pool_idx, 'pool_tick_spacing']
    df['pool_deploy_time'] = pool_info.at[pool_idx, 'pool_deploy_time']
    
    return df


def clean_swap_data(df, pool_info):
    df = clean_data(df, pool_info)
    
    # update dtypes
    df['swap_time'] = pd.to_datetime(df['swap_time'])
    df['pool_deploy_time'] = pd.to_datetime(df['pool_deploy_time'])
    dtypes = {
        'tx_hash': 'string',
        'pool_address': 'string',
        'token_0_amount': float,
        'token_1_amount': float,
        'price_tick': int,
        'price_tick_adjusted': float,
        'contract_address_token_0': 'string',
        'contract_name_token_0': 'string',
        'contract_ticker_symbol_token_0': 'string',
        'contract_decimals_token_0': int,
        'contract_address_token_1': 'string',
        'contract_name_token_1': 'string',
        'contract_ticker_symbol_token_1': 'string',
        'contract_decimals_token_1': int,
        'pool_fee': int,
        'pool_tick_spacing': int
    }
    df = df.astype(dtypes)
    
    return df


def clean_liquidity_data(df, pool_info):
    df = clean_data(df, pool_info)
    df['liquidity_sign'] = df['liquidity_event'].map({
        'ADD_LIQUIDITY': 1,
        'REMOVE_LIQUIDITY': -1
    })
    # pretty sure that all liquidity numbers use 18 decimals, but need to keep
    # an eye on this
    df['liquidity'] = df['abs_liquidity'].map(lambda x: amount_to_float(x, 18))
    df['liquidity'] = df['liquidity'] * df['liquidity_sign']
    df.drop(['abs_liquidity', 'liquidity_sign'], axis=1, inplace=True)
    
    # update dtypes
    df['txn_time'] = pd.to_datetime(df['txn_time'])
    df['pool_deploy_time'] = pd.to_datetime(df['pool_deploy_time'])
    dtypes = {
        'tx_hash': 'string',
        'pool_address': 'string',
        'liquidity_event': 'string',
        'token_0_amount': float,
        'token_1_amount': float,
        'price_tick_lower': int,
        'price_tick_upper': int,
        'price_tick_lower_adjusted': float,
        'price_tick_upper_adjusted': float,
        'contract_address_token_0': 'string',
        'contract_name_token_0': 'string',
        'contract_ticker_symbol_token_0': 'string',
        'contract_decimals_token_0': int,
        'contract_address_token_1': 'string',
        'contract_name_token_1': 'string',
        'contract_ticker_symbol_token_1': 'string',
        'contract_decimals_token_1': int,
        'pool_fee': int,
        'pool_tick_spacing': int,
        'event_name': 'string',
        'liquidity': float,
        'account_address': 'string'
    }
    df = df.astype(dtypes)
    
    return df

In [15]:
pool_info = pd.read_csv('./data/pool_info.csv')
pool_info.info()
pool_info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   index                           8 non-null      int64  
 1   dex_name                        8 non-null      object 
 2   chain_name                      8 non-null      object 
 3   chain_id                        8 non-null      int64  
 4   pool_contract_address           8 non-null      object 
 5   pool_fee                        8 non-null      int64  
 6   pool_tick_spacing               8 non-null      int64  
 7   pool_deploy_time                8 non-null      object 
 8   contract_address_token_0        8 non-null      object 
 9   contract_name_token_0           8 non-null      object 
 10  contract_ticker_symbol_token_0  8 non-null      object 
 11  contract_decimals_token_0       8 non-null      int64  
 12  logo_url_token_0                8 non-nu

,index,dex_name,chain_name,chain_id,pool_contract_address,pool_fee,pool_tick_spacing,pool_deploy_time,contract_address_token_0,contract_name_token_0,...,logo_url_token_0,supports_erc_token_0,contract_address_token_1,contract_name_token_1,contract_ticker_symbol_token_1,contract_decimals_token_1,logo_url_token_1,supports_erc_token_1,n_swaps,n_liquidity
0,5366,uniswap_v3,eth-mainnet,1,0x92c7b5ce4cb0e5483f3365c1449f21578ee9f21a,3000,60,2021-05-06 03:25:59+00:00,0x853d955acef822db058eb8505911ed77f175b99e,Frax,...,https://logos.covalenthq.com/tokens/0x853d955a...,['erc20'],0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,https://logos.covalenthq.com/tokens/0xc02aaa39...,['erc20'],22.0,10.0
1,5508,uniswap_v3,eth-mainnet,1,0x11b815efb8f581194ae79006d24e0d814b7697f6,500,10,2021-05-05 21:46:12+00:00,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,...,https://logos.covalenthq.com/tokens/0xc02aaa39...,['erc20'],0xdac17f958d2ee523a2206206994597c13d831ec7,Tether USD,USDT,6,https://logos.covalenthq.com/tokens/0xdac17f95...,['erc20'],735785.0,6421.0
2,5513,uniswap_v3,eth-mainnet,1,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,500,10,2021-05-05 21:42:11+00:00,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,...,https://logos.covalenthq.com/tokens/0xa0b86991...,['erc20'],0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,https://logos.covalenthq.com/tokens/0xc02aaa39...,['erc20'],1246381.0,31421.0
3,5631,uniswap_v3,eth-mainnet,1,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,3000,60,2021-05-05 19:13:15+00:00,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,Wrapped BTC,...,https://logos.covalenthq.com/tokens/0x2260fac5...,['erc20'],0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,USDC,6,https://logos.covalenthq.com/tokens/0xa0b86991...,['erc20'],59889.0,10913.0
4,5666,uniswap_v3,eth-mainnet,1,0x60594a405d53811d3bc4766596efd80fd545a270,500,10,2021-05-05 18:03:43+00:00,0x6b175474e89094c44da98b954eedeac495271d0f,Dai Stablecoin,...,https://logos.covalenthq.com/tokens/0x6b175474...,['erc20'],0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,https://logos.covalenthq.com/tokens/0xc02aaa39...,['erc20'],295932.0,3515.0
5,5670,uniswap_v3,eth-mainnet,1,0x4e68ccd3e89f51c3074ca5072bbac773960dfa36,3000,60,2021-05-05 16:37:08+00:00,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,...,https://logos.covalenthq.com/tokens/0xc02aaa39...,['erc20'],0xdac17f958d2ee523a2206206994597c13d831ec7,Tether USD,USDT,6,https://logos.covalenthq.com/tokens/0xdac17f95...,['erc20'],159328.0,29299.0
6,5678,uniswap_v3,eth-mainnet,1,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,3000,60,2021-05-04 23:10:00+00:00,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,...,https://logos.covalenthq.com/tokens/0xa0b86991...,['erc20'],0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,https://logos.covalenthq.com/tokens/0xc02aaa39...,['erc20'],194629.0,79804.0
7,5691,uniswap_v3,eth-mainnet,1,0xc2e9f25be6257c210d7adf0d4cd6e3e881ba25f8,3000,60,2021-05-04 20:16:08+00:00,0x6b175474e89094c44da98b954eedeac495271d0f,Dai Stablecoin,...,https://logos.covalenthq.com/tokens/0x6b175474...,['erc20'],0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,https://logos.covalenthq.com/tokens/0xc02aaa39...,['erc20'],85097.0,11042.0


In [16]:
# clean all datasets in the data object
for pool, dset in data.items():
    dset['swaps'] = clean_swap_data(dset['swaps'], pool_info)
    dset['liquidity'] = clean_liquidity_data(dset['liquidity'], pool_info)

In [17]:
with open('./data/pool_data_clean.pickle', 'wb') as f:
    pickle.dump(data, f)